# Bias y Fairness

**2.** Auditaremos `bias` y `fairness` utilizando Aequitas.

Con el modelo que construiste para predecir la edad queremos, como tomadores de decisiones de los hospitales registrados en esta red de Intervenciones Legales del Embarazo, utilizar diferentes métodos anticonceptivos.

A continuación se preparan los datos para su respectiva lectura en `aequitas`. En particular, se incluyen las predicciones del modelo sobre edad mayor o igual a 23 años, `score`, las etiquetas reales de la variable predicha, `label_value`, y la matriz de `features`.

In [92]:
# transformar los datos para importarlos a aequitas.
df_aequitas = pd.DataFrame({'score': pred, 
                        'label_value': y_test_in,
                        'menarca': X_test_in[:,0],
                        'fsexual': X_test_in[:,1],
                        'sememb' : X_test_in[:,2],
                        'nhijos' : X_test_in[:,3],
                        'gesta'  : X_test_in[:,4]})

# guardar la base de datos como csv para hacer analisis online
df_aequitas.to_csv('out_aequitas.csv' , sep=',', index=False)

El toolkit Aequitas nos permite hacer una evaluación de métricas de sesgo y justicia para múltiples grupos de población. 

En este ejemplo en particular, queremos predecir la edad para poder tomar decisiones en hospitales registrados en la red de intervenciones legales del embarazo, con el fin de diagnosticar el uso de diferentes anticonceptivos a la población que permitan prevenir dichas intervenciones. 

Dado lo anterior, la predicción de nuestro modelo servirá para tomar decisiones que afectan la vida de varias mujeres, por lo que debemos tener un análisis detallado de nuestros resultados. 
En particular, para la toma adecuada de anticonceptivos se deben tener en cuenta varios factores, entre ellos la _edad_ y salud de la mujer, efectos secundarios, frecuencia con la que practica relaciones sexuales, etc. Dado que nuestro modelo se centra en predecir si la edad es mayor o igual a 23 años, o no, tomaremos como factor de mayor importancia dicha variable en nuestro análisis.

In [93]:
# implementación en python
import aequitas
from aequitas.preprocessing import preprocess_input_df
df, _ = preprocess_input_df(df_aequitas)

* ¿Qué métricas de bias/fairness calcularás? Justifica tu respuesta.

Las métricas de bias/fairness que queremos calcular dependen del problema del tomador de decisiones. En este caso pertenecemos a los hospitales registrados en esta red de Intervenciones Legales del Embarazo, y suministramos diferentes métodos anticonceptivos a mujeres en función de las predicciones positivas de nuestro modelo.

En particular, para las predicciones que nuestro modelo elegido nos indica que son mayores o tienen 23 años, les pondremos un método anticonceptivo, mientras que para las demás mujeres, correspondeintes a las menores de 23 años, no se decide con el algoritmo, sino que se mira cada caso por específico.

Vamos a plantear las métricas de bias/fairness podrían ser adecuadas en nuestro problema basandonos en el _fairness tree_ de Aequitas, que se presenta abajo.

![FairnessTree](img_results_aequitas/FairnessTree.png)
Fuente: [Aequitas](http://aequitas.dssg.io/)


Para este caso vamos a elegir medidas de _fair/bias_ con base en métricas del error de nuestro sistema. Lo anterior, en la medida que somos tomadores de decisiones de la red de hospitales, y vamos a suministrar anticonceptivos con base en lo que diga nuestro modelo, por lo cual nos interesa saber que tan bueno es su desempeño. El principal problema de tener predicciones desacertadas es que la toma de anticonceptivos puede afectar de manera significativa la salud de las mujeres que realicen la ingesta de los mismos, y podríamos asumir (para este ejercicio), que dicha afectación es importante para menores de 23 años, por ejemplo, a nivel del desarrollo hormonal, ciclos menstruales, fecundidad futura, etc. Por otro lado, se esperaría que dichos efectos no sean significativos en las mujeres de 23 años o más, y por tanto es adecuado que se les proporcionen dichos fármacos. 

Dado lo anterior, quisieramos reducir la cantidad de afectaciones en salud que le causamos a las mujeres para las cuales nuestro modelo predice que tienen 23 años o más, pero esto no es cierto (falso negativo). Por este motivo, vamos a elegir la rama derecha del _fairness tree_ . 

Cabe mencionar que, a modo general, _error-based group metrics_ se concentran en el análisis de errores de tipo I (falsos positivos) y de tipo II (falsos negativos). Dado que estamos haciendo análisis de sesgo y justicia, quisieramos que los errores de estos dos tipos no fueran desbalanceados entre grupos específicos de individuos (parity).

Siguiendo en el segundo nodo del árbol de análisis, debemos ver si nuestras intervenciones son punitivas o de asistencia. En este caso son punitivas, en la medida que podrían afectar negativamente la salud de algunos individuos, si se les suminitran anticonceptivos y no están en la edad adecuada de recibirlos (menores a 23 años). 

Siguiendo lo planteado por [Saleiro, et. al (2019)](https://arxiv.org/pdf/1811.05577.pdf), dado este criterio, la evaluación de bias/fairness se debería basar en las siguientes métricas: _FP Parity_ , _FDR Parity_ , y/o _FPR Parity_. Donde el término _Parity_ evalúa que las distintas métricas sean equilibradas entre los grupos evaluados, para este ejercicio es 80%, y se describe en secciones posteriores.

- $FP_{g}$: Evalúa el número de falsos positivos por cada grupo $g$, es decir, aquellas mujeres que el modelo predijo con 23 o más años, pero en realidad son menores. 
- $FDR_{g}= \frac{FP_{g}}{PP_{g}}$: Evalúa la proporción de falsos positivos del grupo ${g}$ entre el total de predichos positivos en el modelo para el grupo ${g}$, $PP_{g}$.
- $FPR_{g}=\frac{FP_{g}}{LN_{g}}$: Evalúa la proporción de falsos positivos identificados, sobre los etiquetados negativos en el grupo $g$.

Aequitas realiza un análisis global de todas las métricas que aparecen en el  _fairness tree_ . Sin embargo, dado nuestro problema específico, consideraremos el diagrama mencionado y nos centraremos en el cálculo de las dos métricas de las mencionadas anteriormente: FPR y FDR. 

Como línea de base principal, vamos a centrarnos en __FPR__ , es la métrica que más nos interesa, en la medida que nos dice qué tanto se estña equivocando nuestro modelo en predecir si una mujer es menor la 23 años, con relación al total de mujeres con dicha edad, para cada grupo de las variables auditadas.

Para hacer más específico nuestro problema, tenemos que somos miembros de la red de hospitales que realizan intervaneciones legales del embarazo en la Ciudad de México. Dado lo precedente, asumimos que las intervenciones que se realizan en nuestros hospitales miembros trabajan con un porcentaje alto de la población de interés. En dicha medida, y siguiendo el _fairness tree_ , consideramos que nuestra métrica de mayor interés debería ser FPR. Sin embargo, para efectos comparativos de este estudio, también realizaremos análisis de FDR, medida que aplicaría si nuestra red, por ejemplo, fuese pequeña y trabajara con una proporción reducida de la población de interés. Dicha métrica nos ayuda a medir la precisión del modelo.

* ¿Qué variables son las que auditarías? Justifica tu respuesta.

En el análisis de nuestro modelo elegido, decidimos auditar las cinco variables utilizadas en nuestro modelo de clasificación: _sememb_ (semanas de embarazo por fecha de la última menstruación), _nhijos_ (número de hijos), y _gesta_ (número de embarazos, incluyendo abortos), _menarca_ (edad de la primera menstruación), y _fsexual_ (edad de la primera relación sexual). Dada la naturaleza de estas variables, todas son numéricas (discretas), y podríamos querer que nuestras predicciones no estén sesgadas hacia los percentiles que agrupan la mayor parte de la población en cada una de ellas, por lo tanto, nos gustaría dirigir nuestro análisis en esta dirección. 

En dicho sentido, vamos a auditar todas las _features_ incluidas en el modelo, y se va a tomar como grupo de referencia el _Majority group_. Lo anterior, para poder realizar análisis de sesgo y justicia con relación a grupos minoritarios de los individuos considerados. 

Por último, cabe mencionar que dada la naturaleza numérica de las variables, y nuestro desconocimiento previo de las mismas, no encontramos un criterio informado para descartar alguna de las _features_ dentro de nuestro estudio de análisis de sesgo y justicia. 

* Genera el reporte de bias/fairness de Aequitas (en la página web) para los atributos que consideras debemos medir. Utiliza únicamente el set de pruebas. 

    - Incluye imágenes de cada parte del reporte e incluye la interpretación de cada parte: 
        - Métricas por grupo
        - Métricas de disparity
        - Fairness criteria Assessment (con Fair threshold del 80%)

En la presente sección se reporta y estudia el análisis obtenido en la API de Aequitas. Como se observa en la siguiente figura de abajo, se auditaron 12310 filas para las cinco variables numéricas mencionadas con anterioridad. 

El grupo de referencia para cada variable auditada será el grupo que concentra la mayor parte de la población para cada variable. El _Fair threshold_ seleccionado es del 80%.

![Bias_report_MG](img_results_aequitas/Bias_report_MG.png)

Por otro lado, al observar un resumen general (que considera todas las variables y si en su mayoría aprueban o no los criterios) de la auditoria realizada para el modelo, se muestra que en ninguna de las medidas analizadas por el _fairness tree_ cumple con los criterios para considerar que hay justicia y no-sesgo dado el _fair threshold_ de 80%. A continuación se realiza análisis detallado de dichos resultados.

![Bias_summary](img_results_aequitas/Bias_summary.png)

### Audit Results: Details by Fairness Measures

**False Positive Rate Parity**
Como se mencionó anteriormente, considerando _False Positive Rate Parity_ , en el _Fairness tree_ , se puede observar que la decisión en torno a esta métrica dado que nuestra intervención es punitiva, depende de si estamos interviniendo en una proporción muy pequeña de la población de análisis o no. En caso negativo, se recomienda usar FPR, que es un equivalente a la medida de especificidad.

Lo anterior, en la medida que si estamos interviniendo en una proporción grande de la población, quisieramos tener una proporción de error muy baja a la hora de predecir si una mujer tiene menos de 23 años, con respecto al verdadero número de mujeres de dicha edad.
Así, si nuestra especificidad es baja, en las varias intervenciones que tengamos (en un alto % de la población) vamos a tender a fallar en gran medida y vamos a predecir relativamente pocas veces la edad de una menor de 23 años con relación al total de mujeres de dicho grupo -en cada subgrupo evaluado por variable auditada-. Por el contrario, si nuestra especificidad es alta, para las intervenciones que tengamos, tenderemos a fallar menos a la hora de precedir negativos sobre el total de verdaderos negativos.

En pocas palabras, si trabajamos con una gran cantidad de población de mujeres que se realizan una ILE, quisieramos equivocarnos lo mínimo posible a la hora de predecir mujeres menores de 23 años sobre dicho total. Estamos asumiendo que lo anterior es verdadero, ya que pertenecemos a la red de la mayoría hospitales autorizados para realizar estos procedimientos médicos, y también asumimos que recibimos recursos públicos generosos para realizar dicha tarea. 

De igual forma, como mencionamos anteriormente, en este problema nos interesa más no equivocarnos a la hora de predecir que una menor de 23 años no lo es, en la medida que suministrarle un anticonceptivo para el cual su cuerpo no está preparada podría afectar su salud de manera importante.


Ahora bien, si se nos dice que  la red tiene recursos para concentrarse en el 10% de la población, lo natural sería que la precisión en k =10% fuese nuestra métrica a optimizar (y por unidireccionalidad, FDR).

A continuación se muestran los resultados resumidos obtenidos para esta métrica en Aequitas. En particular, FPR, para cada variable, se evalúa la divergencia en la tasa de falsos positivos sobre el número de menroes de 23 años para cada grupo. 

![FPR](img_results_aequitas/FPR.png)

A modo general, la métrica presentada nos es de utilidad en todas las _features_ que se auditan, en la medida que nos gustaría tener tasas de de especificidad(falsos positivos/ negativos etiquetados) similares entre los distintos percentiles de la población, y no favorecer a ninguno. 

Lo precedente se traduciría en tener la misma especificidad en todos los grupos, y por tanto fallar en la misma medida en cada uno de ellos prediciendo menores de 23 años. Sin embargo, dado el análisis realizado, se observa que con respecto a los grupos de referencia mayoritarios, hay sesgo/injusticia en algunos casos. En la siguiente subsección se realiza un análisis detallado de la métrica por variable auditada.

**False Discovery Rate Parity**

La métrica de _FDP_ es análoga a precisión, y la preferiríamos a FPR si estuvieramos interviniendo en un porcentaje pequeño de la población. Así, en el caso ideal,  en las pocas intervenciones que tengamos, vamos a tender a predecir pocos falsos positivos (menores a 23 años predichas como mayores) sobre el total de predichos positivos. Es decir, en dicho escenario tendríamos pocas intervenciones, pero precisas, reduciendo el daño causado a mujeres menores debidos a errores de predicción. 

A continuación se muestra un resumen de dicha métrica, generado con Aequitas.


![FPR](img_results_aequitas/FDR.png)


Al igual que en el caso de FPR, se observa que, para varios grupos de las variables auditadas hay sesgo/injusticia con respecto al grupo mayoritario de comparación. En la próxima sección se realiza de forma breve el análisis a nivel variable. 

### Audit Results: Bias Metrics Values 

En esta sección se presentan de manera individual los resultados para las métricas tanto de FPR como de FDR para cada variable. 

$$medida\:disparidad_{i} = \frac{medida\:disparidad_i}{medida\:disparidad_r}$$

En particular, como elegimos un _fair treshold_ de 80%, de tal forma que, hay sesgo/injusticia para el grupo evaluado $i$, con respecto al grupo mayoritario de referencia $r$ si no se cumple lo siguiente:

$$ 0.80\leq medida\:disparidad_{i}\leq 1.25$$

donde $medida \: disparidad_{i}$ en nuestro caso son FPR y FDR en los grupos analizados y comparados con el grupo de referencia ${r}$.

Es importante anotar que en este ejercicio de comparación el grupo mayoritario, o de referencia tiene medida de disparidad 1. Y por lo tanto estaremos comparando si los grupos minoritarios tienen una medida de disparidad estadísticamente igual a 1 con los criterios utilizados. Dado lo anterior, la el criterio de fairness es:

$$ 0.80\leq medida\:disparidad_i \leq 1.25$$


**Nota:** Para este ejercicio se asumió que si Aequitas define, por ejemplo categorías de grupos para número de hijos como "entre 0 y 1 hijos" y "entre 1 y 2 hijos", el las mujeres con un hijo entran en el último grupo. Lo anterior, dado que no se especifica en la documentación este modelo utilizado en el análisis. 

Por otro lado, no se encontró documentación específica de la forma en que Aequitas clasifica los grupos por cuartiles. Lo que podría ser un impedimento para el análisis de nuestro problema.

- **_menarca_ : Edad de la primera menstruación**
![BMV_menarca](img_results_aequitas/BMV_menarca.png)


Como se puede observar, para la variable de edad de la primera menstruación, el grupo de referencia es "hasta los 12 años", abaracando el 55% de la población (los _group size ratios_ se presentan en la sección de Group metric values) . Y los dos grupos minoritarios son "de 12 a 14 años" (31%), y "entre 14 y 30 años" (14%). 

En torno a FPR, se observa que con respecto al "umbral de justicia" del 80%, el grupo minoritario de "entre 14 y 30 años" presenta sesgo hacia arriba, con una métrica de 1.43. Lo anterior, implica que la proporción de falsos positivos sobre verdaderos negativos es aproximadamente 43% mayor en en el grupo que tuvo su primera mesntruación entre los 14 y 30 años en comparación con el grupo de referencia. En este sentido, dicho sesgo hacia arriba nos haría más proclives a tener a tener una proporción más alta de falsos positivos sobre el total de menores de 23 años en este grupo, y por tanto afectar negativamente a una tasa mayor de mujeres menores a los 23 años.

Por otro lado, el grupo "de 12 a 14 años" tiene una métrica no sesgada, correspondiente a 1.23. En la misma línea, se tiene con respecto al grupo de referencia, la métrica del FPR se encuentra dentro de los umbrales de justicia con respecto al grupo de referencia.

Adicionalmente, con respecto a la métrica del FDR, se observa que en ninguno de los dos grupos minoritarios hay evidencia de sesgo, ubicandose dicha métrica en niveles de 0.92 y 0.89 para los grupos "de 12 a 14 años" y  "entre 14 y 30 años", respectivamente. Esto implica que la proporción de falsos positivios identificados sobre los positivos predichos (mujeres de 23 años o más) es similar entre los grupos de comparación y el grupo mayoritario, sin que haya mejor precisión en uno u otro. 

- **_fsexual_ : Edad de inicio de vida sexual activa**
![BMV_fsexual](img_results_aequitas/BMV_fsexual.png)

Para la variable de edad de inicio de la vida sexual activa, se observa que el grupo mayoritario de referencia es "hasta los 16 años" (41%). Y los grupos minoritarios de comparación son tres categorías adicionales "de 16 a 17 años" (24%), "de 17 a 18 años" (17%), y "entre 18 y 38 años" (18%). 


Con respecto a FPR, la medida de disparidad asociadas para los grupos "de 16 a 17 años", "de 17 a 18 años", y "entre 18 y 38 años", son 1.41, 2.16, y 2.84, respectivamente. 

Dado lo anterior, se encuentra que en todos los grupos minoritarios hay un sesgo hacia arriba en dicha medida, lo que implica que las medidas de FPR asociadas son mayores a las del grupo de referencia. Es decir, que para los grupos minoritarios, tiende a haber una tasa mayor de falso negativos con respecto a los verdaderos negativos. Es decir, que como tomadores de decisiones estaríamos perjudicando a una proporción mayor de menores de 23 años en cada uno de esos subgrupos si comparamos con el grupo mayoritario, un escenario no deseable.

Por otro lado, con relación a FDR, se tiene un sesgo hacia arriba en los grupos "de 16 a 17 años", "de 17 a 18 años", y un sesgo hacia abajo en el grupo "entre 18 y 38 años". Esto implica que en los dos primeros grupos minoritarios, la tasa de falsos positivos (con respecto al total de de predichos positivos) es mayor que en el grupo mayoritario. En este sentido, dicho sesgo hacia arriba implica una peor predicción del modelo en los subgrupos, lo que aumentaría nuestro ratio de mujeres afectadas si consideramos una baja proporción de intervenciones. 

La situación es opuesta para el grupo "entre 18 y 38 años", implicando una situación favorable en términos comparativos apra dicho grupo, ya que hay una mayor precisión en las predicciones. 

- **_sememb_ : Semanas de embarazo por fecha de última menstruación**
![BMV_sememb](img_results_aequitas/BMV_sememb.png)


En cuanto a la variable auditada número de semanas del embarazo a partir de la última menstruación, el grupo mayoritario de referencia corresponde a la categoría "de 0 a 6 semanas", mientras que los grupos minoritarios son "6 a 7 semanas", "de 7 a 9 semanas", y de más de 9 semanas". 

Considerando la métrica de FPR y FDR, en ninguno de los dos casos se observa que haya sesgo para los distintos subgrupos con relación al grupo minoritario. Lo anterior indica que, en términos de estas métricas análogas a especificidad y precisión del modelo, respectivamente, no se encuentran sesgos en las predicciones.

- **_nhijos_ : Número de hijos**
![BMV_nhijos](img_results_aequitas/BMV_nhijos.png)

En cuanto a la variable número de hijos, se encuentra que el grupo de referencia es "de 0 a 1 hijos" (72%), mientras que los grupos minoritarios son "de 1 a 2 hijos" (19%) y "más de 2 hijos". 

En cuanto a la métricas de FPR y  de FDR se encuentra que hay un sesgo hacia arriba de los dos grupos minoritarios con respecto al de referencia. Para nuestro caso de análisis principal se tiene que, con respecto a nuestro grupo de referencia, nuestro modelo tiene una tasa de mujeres menores de 23 años predichas como mayores sobre el total de menores de 23 en los grupos minoritarios. 

Por otro lado, dados los resultados de la FDR, se tiene que la precisión del modelo es menor en cada grupo con respecto al grupo mayoritario. Lo que implica que para los anteriores, nuestra precisión es mucho menor. Y si hicieramos intervenciones para una proporción pequeña de la población, tendríamos una mayor tasa de error en la predicción que naturalemente, terminaría afectando a más mujeres menores de 23 años. 

- **_gesta_ : Número de embarazos (Incluyendo abortos)**
![BMV_gesta](img_results_aequitas/BMV_gesta.png)

Con respecto a la variable número de embarazos (incluyendo abortos), se toma como grupo mayoritario de referencia la categoría "entre 0 y 1" (37%), y los grupos minoritarios a comparar son "1 y 2" (26%), "2 y 3" (20%), y "de 3 a 42" (17%). 

Considerando FPR, se observa que para dicha variable hay sesgo hacia arriba para los tres grupos minoritarios, teniendo que para las categorías "1 y 2 embarazos", "2 y 3 embarazos", y "más de 3" embarazos, su métricas correspondientes son: 5.34, 12.98, y 14.13, respectivamente. Es decir, entre mayor sea el número de embarazos por categoría, el sesgo tiende a incrementar en torno a la relación falsos positivos/etiquetados negativos de cada grupo con respecto al grupo mayoritario, y por tanto, tenderíamos a tener un mayor rango de equivocación que en cada grupo distinto al de referencia afecta a una tasa de %(mujeres menores de 23 años predichas como mayores/ mujeres ) mujeres menores a 23 años. De esta forma, nuestro modelo está siendo injusto con todos los grupos distintos al de referencia.

Por otro lado, considerando FDR, se observa que en dicha medida se tiene un sesgo hacia abajo para los grupos de "2 y 3", y "más de 3" embarazos, con respecto al grupo de referencia. El caso es contrario para el grupo minoritario de entre "1 y 2" embarazos, obteniendo una métrica de 0.95. Así, en el grupo anterior, la relación entre falsos positivos y totales predichos positivos (mayores o iguales a 23 años) es similar al grupo con "entre 0 y 1" embarazos.

### Audit Results: Group Metrics Values 

En el ejercicio realizado anteriormente, dado que la métrica de disparidad del grupo de referencia era siempre 1, se calculaba dicha medida para cada grupo minoritario en sus niveles. Y se comparaba de acuerdo a lo establecido en el criterio de "fairness" definido por la desigualdad definida en la sección anterior.

En el caso de métricas _Group Metrics Values_ , la métrica de disparidad del grupo de referencia no es 1, sino su valor calculado real. En este sentido,  ahora podríamos establecer comparaciones independientes entre el grupo $i$ y el grupo $r$ utilizando el siguiente criterio:

$$medida\:disparidad_{i} = \frac{medida\:disparidad_i}{medida\:disparidad_r}$$

Así, si tomamos como referencia el ejercicio anterior, y comparamos la disparidad de FPR entre el grupo mayoritario  "hasta los 12 años", y el grupo "de 12 a 14 años", obtengo:
$$medida\:disparidad = \frac{0.31}{0.25} = 1.23$$

Lo anterior, es congruente con el resutlado del inciso anterior y nos serviría para comparar múltiples grupos sin tomar necesariamente como referencia el mayoritario.


Por otro lado, dado que identificamos que en general nuestro modelo no es justo cuando tomamos como referencia el grupo mayoritario, nuestra conclusión principal es que es un sistema de predicción débil en este sentido. Para evitar redundancia, a continuación haremos un breve análisis de la métrica de FPR y FDR para cada variable auditada.

Dado que en la sección anterior incluimos el group size ratio en nuestro análisis, lo omitiremos en esta parte. Solo consideraremos la métrica de ${medida\:disparidad_i}$.

- **_menarca_ : Edad de la primera menstruación**
![GM_menarca](img_results_aequitas/GM_menarca.png)

Para la variable auditada de edad de la primera menstruación, con respecto a las métricas de FPR y de FDR, se tiene:
- En la categoría "entre 1 y 12 años" se tiene el menor FPR, 0.25, y el mayor FDR de 0.23 (menor precisión).
- En la categoría "entre 12 y 14 años" se tiene FPR de 0.31, y FDR de 0.21.
- En la categoría "entre 14 y 30 años" se tiene el mayor FPR, 0.36, y FDR de 0.21.
    
Como se mencionó con anterioridad, para nuestra métrica de interés (FPR), se tiene que el grupo mayoritario "entre 1 y 12 años" presenta los menores niveles. Esto implica que en este grupo vamos a tener menos predicciones de mujeres mayores o con 23 años siendo que son menores, con respecto al grupo total de mujeres menores a dicho umbral. 

- **_fsexual_ : Edad de inicio de vida sexual activa**
![GM_fsexual](img_results_aequitas/GM_fsexual.png)

Para la variable auditada de edad de inicio de vida sexual activa, con respecto a las métricas de FPR y de FDR, se tiene:
- En la categoría "entre 0 y 16 años" se tiene el menor FPR, 0.2, y FDR de 0.25.
- En la categoría "entre 16 y 17 años" se tiene FPR de 0.28, y FDR de 0.26.
- En la categoría "entre 17 y 18 años" se tiene FPR, 0.42, y FDR de 0.23.
- En la categoría "Entre 18 y 38 años" se tiene el mayor FPR, 0.56, y el menor FDR, 0.15.
    
Como se mencionó con anterioridad, para nuestra métrica de interés (FPR), se tiene que el grupo mayoritario "entre 0 y 16 años" como presenta los menores niveles. Al igual que en la anterior variable auditada, esto implica que en este grupo vamos a tener menos predicciones de mujeres mayores o con 23 años siendo que son menores, con respecto al grupo total de mujeres menores a dicho umbral. Con respecto a los grupos minoritarios, hay evidencia estadística de imparcialidad, es decir que el grupo de referencia es favorecido (mayor especificidad).

- **_sememb_ : Semanas de embarazo por fecha de última menstruación**
![GM_sememb](img_results_aequitas/GM_sememb.png)

Para la variable auditada de Semanas de embarazo por fecha de última menstruación, con respecto a las métricas de FPR y de FDR, se tiene:

- En la categoría "entre 0 y 6 semanas" se tiene el menor FPR, 0.26, y el menor FDR de 0.21.
- En la categoría "entre 6 y 7 semanas" se tiene FPR de 0.3, y el menor FDR de 0.21.
- En la categoría "entre 7 y 9 semanas" se tiene FPR, 0.28, y FDR de 0.22.
- En la categoría "entre 9 y 49 semanas" se tiene el mayor FPR, 0.29, y FDR, 0.26.

A diferencia de los dos casos anteriores, tenemos que en la métrica del FDR no hay sesgo entre grupos.

- **_nhijos_ : Número de hijos**
![GM_nhijos](img_results_aequitas/GM_nhijos.png)

Para la variable auditada de número de hijos, con respecto a las métricas de FPR y de FDR, se tiene:

- En la categoría "entre 0 y 1 hijos" se tiene el menor FPR, 0.21, y FDR de 0.31.
- En la categoría "entre 1 y 2 hijos" se tiene FPR de 0.94, y FDR de 0.18.
- En la categoría "entre 2 y 9 hijos" se tiene el mayor FPR, 0.95, y el menor FDR, de 0.06.

Con respecto a la métrica de FDR, al igual que en casos anteriores (a excepción de semanas de embarazo), se tiene que al tener la mejor especificidad el grupo mayoritario, este resulta favorecido comparativamente con los grupos minoritarios.

- **_gesta_ : Número de embarazos (Incluyendo abortos)**
![GM_gesta](img_results_aequitas/GM_gesta.png)

Por último, para la variable auditada de número de embarazos, con respecto a las métricas de FPR y de FDR, se tiene:

- En la categoría "entre 0 y 1 nro. de embarazos" se tiene el menor FPR, 0.07, y el mayor FDR de 0.36.
- En la categoría "entre 1 y 2 nro. de embarazos" se tiene FPR de 0.37, y FDR de 0.34.
- En la categoría "entre 2 y 3 nro. de embarazos" se tiene el mayor FPR, 0.91, y FDR, de 0.22.
- En la categoría "entre 2 y 42 nro. de embarazos" se tiene el mayor FPR, 0.99, y el menor FDR, de 0.1.

Con respecto a FDR, se tiene que hay sesgo desfavorable hacia los grupos minoritarios con respecto al grupo de referencia.

- ¿Tu modelo es imparcial? Explica.

De acuerdo al análisis que se realizó en las secciones anteriores, se tiene que a modo general, en todas las variables auditadas (excepto en semanas de embarazo a partir de la última menstruación) el modelo es parcial -injusto- para distintos grupos, considerando dos métricas de error (tanto en FPR como en FDR), de las cuales podríamos fijar una si supieramos los recursos disponibles para resolver el problema. Es decir, si consideramos situaciones en las cuales vamos a tener muchos recursos economómicos e intervenimos con una alta proporción de la población o si la situación es la contraria y queremos tener una precisión muy alta entre grupos, respectivamente. Para este análisis asumimos como dada la primera situación, y por ello nos centramos en su análisis.

La respuesta sobre imparcialidad se debe dar específicamente sobre las variables auditadas. En este sentido, haciendo una breve recapitulación de los resultados en FPR para las variables con sesgo.

Consideraremos imparcialidad desfavorable como el hecho de que para un grupo, el modelo tenga una menor relación entre falsos positivos / total de etiquetados falsos en el grupo. En dicho sentido, para nuestro caso, una menor tasa implica que tenemos menores equivocaciones en porcentaje de mujeres menores a 23 años predichas como mayores con relación al total de menores a 23 años de un grupo. Lo anterior implica un favorecimiento con respecto al grupo con la mejor métrica en la medida que menos mujeres menores a 23 años salen afectadas por el suministro inadecuado de anticonceptivos.

En cuanto a cada variable auditada, se tiene: 

- En _menarca_ y el FPR se encuentra imparcialidad del grupo "entre 14 y 30 años" con respecto al grupo mayoritario "hasta los 12 años". Es decir que hay sesgos favorable hacia el grupo mayoritario. 

- En _fsexual_ , el FPR encuentra imparcialidad para los tres grupos minoritarios con respecto al grupo de referencia "hasta los 16 años". 
- En cuanto a la variable _nhijos_ , el FPR y el FDR indican imparcialidad de los dos grupos minoritarios con respecto al grupo de referencia "de 0 a 1".

- En cuanto a la variable _gesta_, se toma como grupo de referencia "de 0 a 1 embarazos" y tanto FPR como FDR indican sesgo en cuanto a las categorías minoritarias que abarcan mayores números de embarazos. 

    
- ¿En qué atributo tu modelo es lo más unfair, por cuánto?

Para hacer una definición general de _unfair_ "comparable", elegimos las mayores tasas de disparidad entre grupos y las comparamos entre las diferentes variables auditadas para cada métrica. 

Si consideramos el problema que tiene como FPR como métrica principal, se tiene que el modelo es mucho más injusto en la variable auditada gesta. El diferencial (en proporciones) entre el grupo de referencia mayoritario ("de 0 a 1 embarazos") y los grupos minoritarios, son los siguientes:

    - 4.34 para el grupo "de 1 a 2 embarazos" (26% de la población)
    - 13.13 para el grupo "de 2 a 3 embarazos" (20% de la población)
    - 11.98 para el grupo "de más de 2 embarazos" (17% de la población)
    
Cabe mencionar que para esta variable auditada, el grupo mayoritario abarca el 37% de la población. 

- ¿Sugerirías ocuparlo en la realidad? Explica.

No sugerimos utilizar este modelo para ocuparlo en la realidad. Esto, en la medida que presenta sesgos en casi todas las variables auditadas para las dos métricas de error del modelo. 

En este sentido, en casi todos los casos analizados, las minorías están siendo perjudicadas, y en algunos casos favorecidas con respecto al grupo de referencia mayoritario. En un escenario real, quisieramos discriminar a detalle en estos criterios para poder equilibrar a la población mejorando su bienestar. Sin emabargo, cuando nuestras intervenciones son punitivas, y podemos afectar la salud de distintos individuos, quisieramos evitar algún tipo de sesgo en la mayor medida posible.

Por otro lado, en este análisis no se incluyó un análisis de representatibilidad para os diferentes grupos. Es decir, estamos omitiendo si quisieramos que hubiese paridad estadística (igual número de individuos dentro de los grupos de análisis), o representación proporcional. Estos casos son equivalentes a las ramas de la parte izquierda del _fairness tree_ presentado inicialmente. 

- ¿Cuáles son las implicaciones éticas de nos ser imparciales? ¿En qué atributos?

Las implicaciones de son ser imparciales es que directamente estamos favoreciendo o perjudicando por grupos de población, lo que implica que directamente estamos afectando positiva o negativamente vidas de personas. 

En algunos escenarios sería deseable que nuestro modelo tuviera diferentes tipos de sesgos, por ejemplo, en la implementación de programas de políticas públicas. Por ejemplo, en ocasiones se quiere disminuir la desigualdad en términos de riqueza, o mejorar la cantidad y oportunidades de entrar en el mercado laboral de algunos sectores de la población. En estos casos, quisieramos favorecer a los grupos con menos recursos económicos, por ejemplo, si nuestro modelo nos ayudara a decidir a quién se le dan subsidios o no. 

Por otro lado, en el caso que estamos considerando, se considera un tema de salud pública, en el que predecir un positivo puede implicar la afectación de salud de una mujer menor a 23 años. En este sentido, desearíamos que no hubiese sesgos entre los diferentes grupos de la población para las diferentes variables auditadas, caso indeseable que sí ocurre en el análisis de nuestro modelo.

Si consideramos los atributos de nuestro modelo, obtuvimos que para las variables _menarca_ , _fsexual_ , _nhijos_ y _gesta_ , se favorece al grupo mayoritario. En esta medida, dicho grupo tiene una menor proporción de falsos positivos con respecto a los verdaderos negativos en comparación con los grupos minoritarios. Por lo tanto, considerando al considerar dichas _features_ , tenemos que predicciones equivocadas del modelo perjudican a una proporción menor de mujeres si estas pertenecen a la mayoría de cada una de esas variables. Así, sistemáticamente estamos afectando negativamente y en mayor proporción las vidas de las mujeres que se clasifican como "falsos positivos" en los grupos minoritarios.